# LLMRouterChain

In [3]:
import openai
import os 
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_base= os.getenv('OPENAI_API_BASE')
openai.api_type= "azure"
openai.api_version = os.getenv('OPENAI_API_VERSION')
deployment:str=os.getenv('CHATGPT_MODEL')

llm = AzureChatOpenAI(
    deployment_name=deployment,
    model_name=deployment
    )


In [4]:
hr_template="""You are a very smart human resources person. \
You are great at answering questions about human resources in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


In [5]:
law_template="""You are a very smart lawyer. \
You are great at answering questions about the law in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


In [6]:
prompt_infos = [
{
    'name':'human resources',
    'description':'Answer human resources questions',
    'template':hr_template},

{
    'name':'law',
    'description':'Answers legal questions',
    'template':law_template}
]


In [7]:
destination_chains ={}


In [8]:
for p_info in prompt_infos:
    name = p_info['name']
    prompt_template = p_info['template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm,prompt=prompt)
    destination_chains[name] = chain


In [9]:
default_prompt = ChatPromptTemplate.from_template('{input}')

default_chain = LLMChain(llm=llm,prompt=default_prompt)


In [10]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)


Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [11]:

destinations=[
f"{p['name']}: {p['description']}" for p in prompt_infos
]


In [12]:
destinations


['human resources: Answer human resources questions',
 'law: Answers legal questions']

In [13]:
destination_str = "\n".join(destinations)

print(destination_str)


human resources: Answer human resources questions
law: Answers legal questions


In [14]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)

router_template


'Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nhuman resources: Answer human resources questions\nlaw: Answers legal questions\n\

In [15]:
router_prompt = PromptTemplate(template=router_template,
input_variables=['input'],
output_parser=RouterOutputParser())

router_chain = LLMRouterChain.from_llm(llm,router_prompt)

chain = MultiPromptChain(router_chain=router_chain,
destination_chains=destination_chains,
default_chain=default_chain,verbose=True)

chain.run("How do intellectual property laws address the use of AI-generated content?")




> Entering new MultiPromptChain chain...


/Users/jorgeluna/work/learning/langchain/LangChain-Course/venv-langchain-course/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


law: {'input': 'How do intellectual property laws address the use of AI-generated content?'}
> Finished chain.


'Intellectual property laws are still evolving in how they address the use of AI-generated content. Currently, the general rule is that the creator of the AI system that generates the content owns the copyright to that content. However, there may be situations where the user of the AI system could also claim some ownership rights if they made significant creative choices or inputs into the content creation process. Additionally, there are concerns about the potential infringement of existing copyrights if AI-generated content is too similar to existing works. Overall, it is important to seek legal guidance and stay up-to-date on developments in this area as AI technology continues to advance.'

In [16]:
chain.run("How do you handle conflict between team members?")




> Entering new MultiPromptChain chain...


/Users/jorgeluna/work/learning/langchain/LangChain-Course/venv-langchain-course/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


human resources: {'input': 'How do you handle conflict between team members?'}
> Finished chain.


'As an AI language model, I do not have personal experiences, but I can provide some general tips on handling conflicts between team members:\n\n1. Address the issue promptly: Ignoring the conflict will only make it worse. Address it as soon as possible before it escalates.\n\n2. Listen to both sides: Hear both parties out and understand their perspectives. This can help you identify the root cause of the conflict.\n\n3. Remain neutral: As an HR person, it is important to remain neutral and not take sides. Your role is to facilitate a resolution.\n\n4. Encourage open communication: Encourage both parties to openly communicate with each other and express their concerns.\n\n5. Find a mutually beneficial solution: Work with both parties to find a solution that works for everyone. This can involve compromises or finding common ground.\n\n6. Follow up: After the conflict is resolved, follow up with both parties to ensure that the solution is working and that there are no lingering issues.'